# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.62'

In [42]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Oct722")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [43]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [44]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL4-FI1-#1_CRR_DiadFit,103.253808,0.014175,0.010410,1285.320300,0.009076,239.227207,1285.320300,363.144490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL4-FI1-#2 (1)_CRR_DiadFit,103.246802,0.003675,0.003675,1285.353606,0.000000,587.611145,1285.353606,889.332271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL4-FI1-#2_CRR_DiadFit,103.249966,0.005273,0.004054,1285.366563,0.001511,1248.110484,1285.366563,1836.564850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL7-107-FI#2_12mW1acc_CRR_DiadFit,103.611593,0.004321,0.004321,1284.465839,0.000000,2618.741975,1284.465839,5995.685336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL7-109-FI_CRR_DiadFit,103.461170,0.028033,0.028033,1284.821951,0.028033,160.370436,1284.821951,339.343530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [45]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL4-FI1-#1.txt',
 'LL4-FI1-#2 (1).txt',
 'LL4-FI1-#2.txt',
 'LL7-107-FI#1.txt',
 'LL7-107-FI#2_12mW1acc.txt',
 'LL7-107-FI#2_6mW.txt',
 'LL7-109-FI.txt',
 'LL7-FI33.txt',
 'LL7-FI34.txt',
 'LL8-245.txt',
 'LL8-433-FI_12mW.txt',
 'LL8-433-FI_6mW.txt']

## Now get all the important information out of the metadata files

In [46]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 12/12 [00:00<00:00, 117.62it/s]

Done


'LL8-245.txt'

## Then get a simple file name you can stitch with the spectra

In [47]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL8-245'

## Now get filename from spectra into same form as metadata

In [48]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [49]:
file_s

array(['LL4-FI1-#1', 'LL4-FI1-#2 (1)', 'LL4-FI1-#2',
       'LL7-107-FI#2_12mW1acc', 'LL7-109-FI', 'LL7-FI33', 'LL7-FI34',
       'LL8-433-FI_12mW', 'LL8-433-FI_6mW', 'LL7-107-FI#2_6mW',
       'LL7-107-FI#1', 'LL8-245'], dtype='<U21')

In [50]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [51]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,52430.0,0.998239,0.998232,0.998245
1,51844.0,0.998244,0.998238,0.998251
2,51304.0,0.998250,0.998243,0.998256
3,57888.0,0.998202,0.998196,0.998208
4,58966.0,0.998199,0.998193,0.998205


In [52]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [53]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name
0,LL4-FI1-#1.txt,1150.775772,84.693737,60.851251,Spline
1,LL4-FI1-#2 (1).txt,1150.716797,93.719106,67.561877,Spline
2,LL4-FI1-#2.txt,1150.746284,169.521071,137.106448,Spline


In [54]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [55]:
file_sec_phase

array(['LL4-FI1-#1', 'LL4-FI1-#2 (1)', 'LL4-FI1-#2'], dtype=object)

In [56]:
df_combo['name_for_matching']

0                LL4-FI1-#1
1            LL4-FI1-#2 (1)
2                LL4-FI1-#2
3     LL7-107-FI#2_12mW1acc
4                LL7-109-FI
5                  LL7-FI33
6                  LL7-FI34
7           LL8-433-FI_12mW
8            LL8-433-FI_6mW
9          LL7-107-FI#2_6mW
10             LL7-107-FI#1
11                  LL8-245
Name: name_for_matching, dtype: object

In [57]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [58]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,duration,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,SO2_Diad_Ratio
0,0,103.071932,103.082343,103.061520,0.010411,LL4-FI1-#1_CRR_DiadFit,103.253808,0.014175,0.010410,1285.320300,...,"['0h', '3m', '46s']",2:33:50 PM\n,52430.0,1324.987,LL4-FI1-#1.txt,1150.775772,84.693737,60.851251,Spline,0.091963
1,0,103.065530,103.069256,103.061804,0.003726,LL4-FI1-#2 (1)_CRR_DiadFit,103.246802,0.003675,0.003675,1285.353606,...,"['0h', '3m', '46s']",2:24:04 PM\n,51844.0,1324.987,LL4-FI1-#2 (1).txt,1150.716797,93.719106,67.561877,Spline,0.041808
2,0,103.069266,103.073367,103.065166,0.004101,LL4-FI1-#2_CRR_DiadFit,103.249966,0.005273,0.004054,1285.366563,...,"['0h', '3m', '46s']",2:15:04 PM\n,51304.0,1324.987,LL4-FI1-#2.txt,1150.746284,169.521071,137.106448,Spline,0.037139
3,0,103.425332,103.429691,103.420973,0.004359,LL7-107-FI#2_12mW1acc_CRR_DiadFit,103.611593,0.004321,0.004321,1284.465839,...,"['0h', '1m', '23s']",4:04:48 PM\n,57888.0,1324.987,NaN,NaN,NaN,NaN,NaN,NaN
4,0,103.274818,103.302808,103.246828,0.027990,LL7-109-FI_CRR_DiadFit,103.461170,0.028033,0.028033,1284.821951,...,"['0h', '3m', '47s']",4:22:46 PM\n,58966.0,1324.987,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [60]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()